In [147]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr
import math
from scipy.stats import wasserstein_distance

In [280]:
Predicted = pd.read_csv("./Seurat/Outputs/output_Sim29.csv")
Target=pd.read_csv("./Data/Sim29/target.csv")

In [281]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
#Predicted.drop(columns="cell_ID", inplace=True)
Predicted.drop(columns="prediction.score.max",inplace=True)


In [282]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [283]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [284]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [285]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [286]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [287]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [288]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [289]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [290]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

In [291]:
np.mean(np.nan_to_num(d))

0.6241022594974965

In [292]:
d

array([ 0.91524815,  0.88650374,  0.62501776,  0.60247131,  0.6564814 ,
        0.4115715 ,  0.5891921 , -0.00424659,  0.9051214 ,  0.65366182])

## Jenson Shannon Divergence

In [293]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.44288182415984056

## F1 Score, Precision, Recall

In [294]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [295]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [296]:
d

,Precision,Recall,F1-score
0.100,0.830789,0.467933,0.598671
0.200,0.767144,0.464204,0.578409
0.300,0.630584,0.478176,0.543905
0.010,0.811159,0.581936,0.677689
0.001,0.788369,0.616832,0.692131


## Wasserstein Distance

In [297]:
scipy.stats.wasserstein_distance(np.reshape(Target,(Target.shape[0]*Target.shape[1])),np.reshape(Predicted,(Predicted.shape[0]*Predicted.shape[1])))

0.06040359756812254

## KL Divergence

In [298]:
to_be_kept=(Predicted>0)

In [299]:
Predicted=Predicted[to_be_kept]
Target=Target[to_be_kept]

In [300]:
kl=scipy.special.rel_entr(Target,Predicted)

In [301]:
np.mean(kl)

0.12403212772956108